MSNN : https://github.com/Nagendra1693/MSNN/blob/master/MSNN%20V1.0.ipynb   

---

HSNN : https://github.com/amFahad/Cuda_HSNN/blob/main/HSNN%20V1.0.ipynb

---


Both these Models are combines below for classifying Scene-15 dataset. data is passed through MSNN and features extracted from this model is given to HSNN for Final classification.



In [ ]:
!pip install pycuda
!pip install scikit-cuda

     |████████████████████████████████| 1.6MB 9.1MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp37-cp37m-linux_x86_64.whl size=621490 sha256=c676fca2a1e998d98035d96cdf9cb9be3a8893be384e11bb88c99ebf86c68481
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2021.2.1-py2.py3-none-any.whl size=62786 sha256=1aff689b0d5deaace2f15300a9974dbc33fb7e99dcdc3776410f004a387bf630
  Stored in directory: /root/.cache/pip/wheels/61/7c/29/7c3f742ab75f8f2df376b72b43e6640167ffbf668c950429bc
Successfully built pycuda pytools
     |████████████████████████████████| 122kB 8.2MB/s 


In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np
import math
import scipy
import random
from sklearn.metrics import mean_squared_error
import pickle as pkl
from sklearn.preprocessing import MinMaxScaler
from tqdm import trange
import sys
import os
import cv2
from sklearn.preprocessing import normalize
import time

import pycuda.autoinit
import pycuda.driver as cuda
import pycuda.gpuarray as gpua
import pycuda.curandom
import pycuda.driver as drv
from pycuda.compiler import SourceModule
from pycuda import cumath

import skcuda
import skcuda.linalg as linalg
linalg.init()


physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [ ]:
# Matrix Inverse
def matrix_inverse(A, cuda = True, coefficient = None):
    m, n = A.shape
    transpose = False
    pinv = None
    A = np.ascontiguousarray(A)
#     A is m-by-n and the rank of A is equal to m (m ≤ n) then A has right inverse A_P = At * inv(A * At)
#     A is m-by-n and the rank of A is equal to n (n ≤ m) then A has left inverse A_P = A * inv(At * A)

    if m > n:
        A = np.ascontiguousarray(A.T)
        transpose = True
# By default this calculates the right inverse
    At = np.ascontiguousarray(A.T)
    if cuda:
        A_gpu = gpua.to_gpu(A)
        At_gpu = gpua.to_gpu(At)
        
        if coefficient is None:
            inv = linalg.inv(linalg.dot(A_gpu, At_gpu))
        else:
            inv = linalg.inv( gpua.to_gpu(np.identity(A.shape[0])/coefficient) + linalg.dot(A_gpu, At_gpu)).get()
        #pinv=At.dot(inv)
        pinv = linalg.dot(At_gpu, inv).get()
    else:
        inv = np.linalg.inv(A.dot(At))
        pinv = At.dot(inv)
        
    if transpose:
        pinv = pinv.T
        
    return pinv

def exp(x):
    x_gpu = gpua.to_gpu(x)
    x_exp = cumath.exp(x_gpu).get()
    return x_exp

#maxpooling
def maxpool(A,B):
  C=A
  for i in range(len(C)):
    for j in range(len(C[0])):
      if(A[i][j] > B[i][j]):
        C[i][j] = A[i][j]
      else:
        C[i][j] = B[i][j]
  return C



In [ ]:

class MSNN():
    def __init__(self):        
        self.af = []
        self.bf = []
        
        self.hf = []
        
        self.scalers = []
        self.act_fn = None
    
    def exp(self, x):
        x_gpu = gpua.to_gpu(x)
        x_exp = cumath.exp(x_gpu).get()
        return x_exp
    
    def act(self, h):
        if self.act_fn == 'sigmoid':
            h = 1/ (1+ self.exp(-h))
        elif self.act_fn == 'sine':
            h = np.sin(h)
        return h

    def act_inv(self, h):
        if self.act_fn == 'sigmoid':
            h = -1 * np.log((1/h) -1)
        elif self.act_fn == 'sine':
            h = np.arcsin(h)
        return h
    
    def fit(self,x, y, d, l_max, act_fn):
        self.act_fn = act_fn
        
        instances = x.shape[0]
        features = x.shape[1]
        h_saved = None
        y_scaler = MinMaxScaler(feature_range=(0.0001, 0.9999))
        p_scaler = MinMaxScaler(feature_range=(0.0001, 0.9999))
        
        x_inv = matrix_inverse(x, True)
        
        for j in trange(l_max, file=sys.stdout):
#             Step1
            if j == 0:
                af = np.random.normal(size = (features, d)) # (d, 784)
                bf = np.random.normal(size = (1, 1))
                
                h = self.act(np.dot(x, af) + bf)
#                 print('Step 1 is done')
#             Step4
            else:                
                pj_norm = self.act_inv(p_scaler.fit_transform(Pj) + 0j).real
            
                af = np.dot(x_inv, pj_norm)
                bf = mean_squared_error(pj_norm, Pj, squared=False)
                
                h = self.act(np.dot(x, af) + bf)
                h += p_scaler.inverse_transform(h)
#                 print('Step 4 is done')
  
            self.af.append(af)
            self.bf.append(bf)
            self.scalers.append(p_scaler)
            
#             Step2
            y_norm = self.act_inv(y_scaler.fit_transform(y) + 0j).real
            ah = np.dot(np.linalg.pinv(h), y_norm)
            bh = mean_squared_error(np.dot(h, ah), y_norm, squared=False)

#             print('Step 2 is done')
#             Step3
            elm_h = self.act(np.dot(h, ah) + bh)

            ej = y - y_scaler.inverse_transform(elm_h)
            e_norm = self.act_inv(y_scaler.transform(ej) + 0j).real
            
            Pj = np.dot(e_norm, np.linalg.pinv(ah))
#             print('Step 3 is done')


            
    def transform(self, x):
        if len(self.af) == 0:
            raise 'There is no feature mapping model'
            
        feature_data = None
        for i in range(len(self.af)):
            h = self.act(np.dot(x, self.af[i]) + self.bf[i])  
            if feature_data is None:
                feature_data = h
            else:
                h = self.scalers[i].inverse_transform(h)
                feature_data += h
        return feature_data
    
    def fit_transform(self, x, y, d, l_max, act_fn):
        self.fit(x, y, d, l_max, act_fn)
        return self.transform(x)
        

In [ ]:
import random
class ELM:
    def __init__(self, hidden_nodes, activation):
        self.features = None
        self.hidden_nodes = hidden_nodes
        self.activation = activation

        self.input_weights = None
        self.output_weights = None  # beta
        self.bias = None

    def hidden_layer(self, x):
        bias = np.array([self.bias[0],]* x.shape[0])
        h = np.dot(x, self.input_weights) + bias
        
        h = h + 0j
        
        if self.activation == 'relu':
            h = np.maximum(h, 0, h)
        elif self.activation == 'sigmoid':
            h = 1 / (1 + np.exp(-h))
        elif self.activation == 'sine':
            h = np.sin(h)
        return h.real

    def fit(self, x_train, y_train):
        self.features = x_train.shape[1]
        self.input_weights = np.random.normal(size=(x_train.shape[1], self.hidden_nodes))
        self.bias = np.random.random((1, self.hidden_nodes))
        H = self.hidden_layer(x_train)
        self.output_weights = np.dot(np.linalg.pinv(H), y_train)

        pred = self.predict(x_train)
        acc = self.performance(y_train, pred)
        return pred, acc

    def predict(self, x):
        H = self.hidden_layer(x)
        predictions = np.dot(H, self.output_weights)
        return predictions

    def evaluate(self, x_test, y_test):
        pred = self.predict(x_test)
        acc = self.performance(y_test, pred)
        return pred, acc

    def performance(self, y_actual, y_predicted):
        y_actual = np.argmax(y_actual, axis=-1)
        y_predicted = np.argmax(y_predicted, axis=-1)

        correct = np.sum(y_actual == y_predicted)
        accuracy = (correct / y_actual.shape[0]) * 100
#         accuracy = format(accuracy, '.2f')
        return accuracy

In [ ]:

train = np.genfromtxt('train.csv', delimiter=',')
test = np.genfromtxt('test.csv', delimiter=',')
T = train.T[0][None]
OrgT=T;
P = train.T[1:]
OrgP=P;
TV_T = test.T[0][None]
OrgTT=TV_T;
TV_P = test.T[1:]
NumberofTrainingData=len(P[1])
NumberofTestingData=len(TV_P[1])
NumberofInputNeurons=len(P);
NumberofOutputNeurons=len(np.unique(T));        
#ONE HOT ENCODING
Temp_T = np.zeros((NumberofOutputNeurons,NumberofTrainingData))
count = 0
for i in T.T:
  Temp_T[int(i[0]-1)][count] = 1
  count+=1
T = Temp_T*2-1
count = 0
Temp_T = np.zeros((NumberofOutputNeurons,NumberofTestingData))
for i in TV_T.T:
  Temp_T[int(i[0]-1)][count] = 1
  count+=1
TV_T = Temp_T*2-1
(x_train, y_train), (x_test, y_test) = (P.T,T.T),(TV_P.T,TV_T.T)

In [ ]:
x_train_m = x_train
# x_train_m = np.array(x_train, dtype=np.float64)
y_train_m = y_train
x_test_m = x_test
y_test_m = y_test

m = MSNN()

m.fit(x = x_train_m, y = y_train_m, d = 1500, l_max = 2, act_fn = 'sigmoid')
x_train_new = m.transform(x_train_m)
x_test_new = m.transform(x_test_m)

elm = ELM(hidden_nodes = 100, activation = 'relu')

_, train_acc = elm.fit(x_train_new, y_train)
_, test_acc = elm.evaluate(x_test_new, y_test)
print('\tTraining Accuracy : ', train_acc)
print('\tTesting Accuracy : ', test_acc)

100%|██████████| 2/2 [00:05<00:00,  2.63s/it]
	Training Accuracy :  61.0
	Testing Accuracy :  55.27638190954774


In [ ]:
import numpy as np
import sys
import os
import cv2
from sklearn.utils import check_random_state
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import normalize, MinMaxScaler
from sklearn.metrics.pairwise import polynomial_kernel
import math
import time
import tensorflow as tf

NumberofHiddenNeurons=100
C=256
kkkk=2
sn=3
name='scene15_channel_1'

def subnetwrork(P, TV_P, T, TV_T, NumberofHiddenNeurons, C, kkkk, sn, name):
    fdafe=0
    fdafe1=0
    asin = np.vectorize(math.asin)
    sin = np.vectorize(math.sin)
    #Creating testing and trainging data
    OrgT=T;
    OrgP=P;
    OrgTT=TV_T;
    NumberofTrainingData=len(P[1])
    NumberofTestingData=len(TV_P[1])
    NumberofInputNeurons=len(P);
    NumberofOutputNeurons=len(np.unique(T));
    saveT=T
    
    for subnetwork in range(1,sn+1):
        for j in range(1,kkkk+1):
            if(j==1):
                BiasofHiddenNeurons1 = np.random.rand(NumberofHiddenNeurons)
                BiasofHiddenNeurons1 = normalize(BiasofHiddenNeurons1[None], norm='l2')
                InputWeight1=np.random.rand(NumberofHiddenNeurons,NumberofInputNeurons)*2-1
                InputWeight1 = normalize(InputWeight1, norm='l2')
                
                tempH=InputWeight1.dot(P);
                ind=np.ones((1,NumberofTrainingData));
                BiasMatrix=ind * BiasofHiddenNeurons1.T
                tempH=tempH+BiasMatrix;
            else:
                PP = np.where(PP > 1, 1, PP)
                PP = np.where(PP < -1, -1, PP)
                YYM_PP=PP
                PP1=asin(PP)
                PP1=PP1.real
                P=P_save
                InputWeight1=matrix_inverse(P, cuda = True, coefficient = C).T.dot(PP1.T)
                #InputWeight1=np.linalg.lstsq((np.identity(len(P))/C+P.dot(P.T)),P)[0].dot(PP1.T)
                InputWeight1 = InputWeight1.T
                fdafe=0
                tempH=InputWeight1.dot(P)
                PTV_P = np.concatenate((P,TV_P),axis=1)
                TTV_T = np.concatenate((saveT,TV_T),axis=1)
                YYM_H = InputWeight1.dot(PTV_P)
                BB1=np.array(PP1.shape)
                BB2=sum(sum(tempH-PP1))
                BBP=BB2/BB1[1]
                tempH = (tempH.T - BBP).T
                YYM_tempH = (YYM_H.T - BBP).T
                
            
            H = sin(tempH)
            
            if(j>1):
                YYM_H = sin(YYM_tempH)
                scaler2 = MinMaxScaler(feature_range=(-1,1))
                YYM_H = scaler2.fit_transform(YYM_H)
                H = YYM_H.T[0:NumberofTrainingData].T
                fname = "scene15_channel_1feature_"+str(subnetwork)+".csv"
                np.savetxt(fname, YYM_H, delimiter=",")
                np.savetxt("scene15_channel_1target.csv", TTV_T, delimiter=",")
                
            P_save=P;
            P=H;
            FT=np.zeros((3,17766))
            E1=T
            TrainingAccuracy = []
            for i in range(1,3):
                a=0.0000001;
                Y2=E1;
                
                if(fdafe==0):
                    scaler = MinMaxScaler(feature_range=(-1,1))
                    Y22 = scaler.fit_transform(Y2) #scaler.inverse_transform(X)
                else:
                    scaler3 = MinMaxScaler(feature_range=(-1,1))
                    Y22 = scaler3.fit_transform(Y2)
                
                Y22 = np.where(Y22 > 1, 1, Y22)
                Y22 = np.where(Y22 < -1, -1, Y22)
                Y2=Y22                
                Y4=asin(Y2)
                Y4=Y4.real
                
                if(fdafe==0):
                    YYM = matrix_inverse(P.T, cuda = True, coefficient = C).dot(Y4.T)
                    #YYM = np.linalg.lstsq((np.identity(len(P))/C+linalg.dot(P_gpu,PT_gpu).get()),P)[0].dot(Y4.T)
                    YJX=YYM.T.dot(P).T;
                else:
                    PP = Y4.T.dot(matrix_inverse(YYM, cuda = True, coefficient = C)).T
                    #PP = Y4.T.dot(np.linalg.lstsq(  (np.identity(len(YYM))/C+YYM.dot(YYM.T)),YYM)[0].T).T
                    YJX=PP.T.dot(YYM);
                    
                BB1=np.array(Y4.shape);
                BB2=sum(YJX-Y4.T)
                BB=BB2/BB1[1];
                BB=BB[0]
                GXZ111 = (YJX.T - BB).T
                GXZ2=sin(GXZ111.T)
                
                FYY = scaler.inverse_transform(GXZ2).T
                FT1 =  FYY.T
                E1=E1-FT1
                if(i==1):
                    FT=FT1
                    fdafe=1
                else:
                    FT=FT+FT1
                
                if(subnetwork==1):
                    FTY=FT;
                    FTY_prev = FTY
                else:
                    FTY=FTY_prev-FT;
                    
                MissClassificationRate_Training=0;
                for idx in range(len(saveT[0])):
                    label_index_expected = np.where(saveT.T[idx] == np.amax(saveT.T[idx]))
                    label_index_actual = np.where(FTY.T[idx] == np.amax(FTY.T[idx]))
                    if(label_index_expected!=label_index_actual):
                        MissClassificationRate_Training+=1
                
                TrainingAccuracy.append(1-MissClassificationRate_Training/len(saveT[0]))
                 
            PP=PP+P;
        
        T=E1
        P=OrgP
        outputweight=YYM
        fdafe=0
    i=1

def featurecomb(sn):
    for i in range(1,sn+1):
        fname = "scene15_channel_1feature_"+str(i)+".csv"
        if(i==1):
            Features = np.genfromtxt(fname, delimiter=',')
        elif(i==2):
            Features = Features + np.genfromtxt(fname, delimiter=',')
    return Features

def lastlayer(Features,Target):
    C2=4096
    kkk=1
    P = Features[:,0:1500]
    T = Target[:,0:1500]
    TV_P = Features[:,1500:]
    TV_T = Target[:,1500:]
    NumberofTrainingData=len(P[1])
    NumberofTestingData=len(TV_P[1])
    NumberofInputNeurons=len(P)
    E1=T
    
    Y2=E1
    scaler = MinMaxScaler(feature_range=(0.01,0.99))
    Y22 = scaler.fit_transform(Y2)
    scaler2 = MinMaxScaler(feature_range=(0.01,0.99))
    chumma = scaler2.fit_transform(TV_T)
    Y2=Y22.T
    def sigmoid(x):
    	# x = -x
        # return 1 / (1 + exp(-x))
        return 1 / (1 + math.exp(-x))
    sigmoid = np.vectorize(sigmoid)
    Y4=sigmoid(Y2).T
    YYM=matrix_inverse(P, cuda = True, coefficient = C).T.dot(Y4.T)
    #YYM=np.linalg.lstsq((np.identity(len(P))/C2+P.dot(P.T)),P)[0].dot(Y4.T)
    YJX=P.T.dot(YYM)
    BB1=np.array(Y4.shape)
    BB2=sum(YJX-Y4.T)
    BB=BB2[0]/BB1[1]
    GXZ111=(P.T.dot(YYM))-BB
    GXZ2=sigmoid(GXZ111).T
    FYY = scaler.inverse_transform(GXZ2)
    FT1=FYY
    TrainingAccuracy=math.sqrt(mse(E1,FT1))
    E1=E1-FT1
    Y=FYY
    D_YYM=YYM
    MissClassificationRate_Training=0
    for idx in range(len(T[0])):
        label_index_expected = np.where(T.T[idx] == np.amax(T.T[idx]))
        label_index_actual = np.where(Y.T[idx] == np.amax(Y.T[idx]))
        if(label_index_expected!=label_index_actual):
            MissClassificationRate_Training+=1
    TrainingAccuracy=(1-MissClassificationRate_Training/len(T[0]))*100
    print("TrainingAccuracy",TrainingAccuracy)
    
    GXZ1=D_YYM.T.dot(TV_P-BB)
    GXZ2=sigmoid(GXZ1).T
    FYY = scaler2.inverse_transform(GXZ2.T)
    TY2=FYY
    E1=TY2-TV_T
    TestingAccuracy=math.sqrt(mse(TY2,TV_T))
    MissClassificationRate_Testing=0
    for idx in range(len(TV_T[0])):
        label_index_expected = np.where(TV_T.T[idx] == np.amax(TV_T.T[idx]))
        label_index_actual = np.where(TY2.T[idx] == np.amax(TY2.T[idx]))
        if(label_index_expected!=label_index_actual):
            MissClassificationRate_Testing+=1
    TestingAccuracy=(1-MissClassificationRate_Testing/len(TV_T[0]))*100
    print("TestingAccuracy",TestingAccuracy)

start_time = time.time()     
subnetwrork(x_train_new.T, x_test_new.T, y_train.T, y_test.T, NumberofHiddenNeurons, C, kkkk, sn, name)
print("Subspace Feature Extraction Time : ",time.time()-start_time,"\n")

start_time = time.time() 
Features = featurecomb(sn)
print("Combining Feature Time Taken : ",time.time()-start_time,"\n")

Target = np.genfromtxt("scene15_channel_1target.csv", delimiter=',')

start_time = time.time()
lastlayer(Features,Target)
print("Final Classification Time Taken : ",time.time()-start_time)


Subspace Feature Extraction Time :  3.4086523056030273 

Combining Feature Time Taken :  0.9227828979492188 

TrainingAccuracy 100.0
TestingAccuracy 90.98827470686768
Final Classification Time Taken :  0.17957210540771484
